In [1]:
# Libraries

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.losses import Loss
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, DepthwiseConv2D, Bidirectional,
    BatchNormalization, ReLU, Add, Dropout, LSTM, MultiHeadAttention, Reshape,Multiply)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, roc_curve, precision_recall_curve, classification_report
)
import tensorflow as tf
import shap
from lime import lime_image
from skimage.segmentation import mark_boundaries
from tf_explain.core.grad_cam import GradCAM
import json

In [ ]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [4]:
# Data Loading and Preprocessing

In [5]:
train_dir = r'D:\Multi-Class Diabetic Retinopathy Classification\data\aptos2019-blindness-detection\dataset\train'
test_dir = r'D:\Multi-Class Diabetic Retinopathy Classification\data\aptos2019-blindness-detection\dataset\test'

In [6]:
# Image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    rotation_range=180,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    zoom_range=0.2,  
    shear_range=0.2,  
    brightness_range=[0.8, 1.2],  
    horizontal_flip=True,  
    fill_mode='constant',  
    cval=0,  
    validation_split=0.2  
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Load and preprocess the training and testing images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use 'training' subset for training data
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use 'validation' subset for validation data
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
img_batch, _ = next(train_generator)  
file_paths = train_generator.filepaths
image_path = file_paths[train_generator.index_array[0]]  
original_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)  

category = image_path.split(os.sep)[-2]

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1), plt.imshow(original_img), plt.title("Original Image"), plt.axis('off')
plt.subplot(1, 3, 2), plt.imshow(img_batch[0]), plt.title("Processed Image"), plt.axis('off')
plt.subplot(1, 3, 3), plt.text(0.5, 0.5, f"Label: {category}", fontsize=12, ha='center', va='center'), plt.title("Category"), plt.axis('off')
plt.tight_layout(), plt.show()


In [9]:
# Model Definition

In [10]:
def residual_block(x, filters, stride=1):
    # Shortcut
    shortcut = x
    
    # Conv Layer 1
    x = Conv2D(filters, kernel_size=3, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Conv Layer 2
    x = Conv2D(filters, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    # Shortcut Projection (if necessary)
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    # Add shortcut to the main path
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

In [11]:
input = Input(shape=(224, 224, 3))
x = Conv2D(64, kernel_size=7, strides=2, padding='same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

x = residual_block(x, filters=64, stride=1)
x = residual_block(x, filters=64, stride=1)
x = residual_block(x, filters=128, stride=2)
x = residual_block(x, filters=128, stride=1)
x = residual_block(x, filters=256, stride=2)
x = residual_block(x, filters=256, stride=1)
x = residual_block(x, filters=512, stride=2)
x = residual_block(x, filters=512, stride=1)

x = GlobalAveragePooling2D()(x)
x = Reshape((1, -1))(x)

x = MultiHeadAttention(num_heads=4, key_dim=x.shape[-1])(x, x)
x = tf.squeeze(x, axis=1)
x = Reshape((1, -1))(x)

x = Bidirectional(LSTM(64, return_sequences=False))(x)
x = Dense(128, activation='relu')(x)
output = Dense(5, activation='softmax')(x)


In [ ]:
model = Model(inputs=input, outputs=output)

model.summary()

In [ ]:
# The weighted cross-entropy loss function
class ImprovedFocalLoss(tf.keras.losses.Loss):
    def __init__(self, weights=None, gamma=2.0, label_smoothing=0.1, **kwargs):
        super().__init__(**kwargs)
        self.weights = tf.constant(weights, dtype=tf.float32) if weights is not None else None
        self.gamma = gamma
        self.label_smoothing = label_smoothing

    def call(self, y_true, y_pred):
        y_pred = tf.cast(y_pred, tf.float32)
        y_true = tf.cast(y_true, tf.float32)

        y_true = y_true * (1 - self.label_smoothing) + self.label_smoothing / tf.cast(tf.shape(y_true)[-1], tf.float32)

        y_pred = tf.keras.backend.clip(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())

        cross_entropy = -y_true * tf.math.log(y_pred)

        focal_factor = tf.pow(1 - y_pred, self.gamma)

        if self.weights is not None:
            weights = tf.cast(self.weights, y_pred.dtype)  
            loss = tf.reduce_sum(weights * focal_factor * cross_entropy, axis=-1)
        else:
            loss = tf.reduce_sum(focal_factor * cross_entropy, axis=-1)

        return loss


train_labels = train_generator.classes
num_samples_per_class = np.bincount(train_labels)
class_weights = 1.0 / (num_samples_per_class + 1e-6)
class_weights = class_weights / np.sum(class_weights)

focal_loss_fn = ImprovedFocalLoss(weights=class_weights, gamma=2.0, label_smoothing=0.1)
optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss=focal_loss_fn, metrics=['accuracy'])


In [14]:
# Model Training

In [15]:
checkpoint = ModelCheckpoint('model_best.h5',  
                             monitor='val_loss',
                             save_best_only=True, 
                             mode='min') 

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=15, 
                               mode='min',
                               restore_best_weights=True) 

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2, 
                              patience=5,  
                              min_lr=0.00001,
                              mode='min')


In [ ]:

history = model.fit(
    train_generator,  
    steps_per_epoch=train_generator.samples // train_generator.batch_size, 
    validation_data=validation_generator,  
    validation_steps=validation_generator.samples // validation_generator.batch_size, 
    epochs=15,  
    callbacks=[checkpoint, early_stopping, reduce_lr],
    workers=4,  
    max_queue_size=10)

In [36]:
# Model Evaluation

In [37]:
save_dir='plots'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [ ]:
# Plot the training and validation loss and accuracy curves
plt.figure(figsize=(12, 5))
# Accuracy curves
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, 'accuracy_plot.png'), dpi=600, bbox_inches='tight')
plt.show()
# Plot the training and validation loss and accuracy curves
plt.figure(figsize=(12, 5))
# Loss curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, 'loss_plot.png'), dpi=600, bbox_inches='tight')
plt.show()

In [39]:
# Model Loading

In [40]:
model = load_model('model_best.h5', custom_objects={'FocalLoss': ImprovedFocalLoss})

In [41]:
# Prediction and Inference

In [42]:
# Make predictions on the test data
predictions = model.predict(test_generator)  
true_labels = test_generator.classes  
class_labels = list(test_generator.class_indices.keys())  
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_classes)
def evaluate_model(true_labels, predicted_classes, class_labels, save_dir):
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    
    overall_accuracy = accuracy_score(true_labels, predicted_classes)
    
    # Generate classification report
    classification_report_dict = classification_report(
        true_labels, predicted_classes, target_names=class_labels, output_dict=True, zero_division=0
    )

    # Calculate specificity and accuracy per class
    specificity_per_class = []
    accuracy_per_class = []
    total_tn, total_fp = 0, 0  # For micro-average specificity
    for i in range(len(class_labels)):
        true_negative = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
        false_positive = np.sum(cm[:, i]) - cm[i, i]
        specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0.0
        specificity_per_class.append(specificity)

        accuracy = cm[i, i] / np.sum(cm[i, :]) if np.sum(cm[i, :]) > 0 else 0.0
        accuracy_per_class.append(accuracy)

        total_tn += true_negative
        total_fp += false_positive

    # Calculate macro and micro average specificity
    macro_specificity = np.mean(specificity_per_class)
    micro_specificity = total_tn / (total_tn + total_fp) if (total_tn + total_fp) > 0 else 0.0

    # Save classification report as JSON
    json_save_path = os.path.join(save_dir, "classification_report.json")
    with open(json_save_path, "w") as f:
        json.dump(classification_report_dict, f, indent=4)
    print(f"Classification Report saved to {json_save_path}")

    # Print classification report
    print("Classification Report:")
    print(f"{'Class':<15} {'Precision':<8} {'Recall':<8} {'F1-Score':<8} {'Specificity':<12} {'Accuracy':<8} {'Support':<8}")
    for idx, cls in enumerate(class_labels):
        metrics = classification_report_dict[cls]
        print(f"{cls:<15} {metrics['precision']:<8.2f} {metrics['recall']:<8.2f} {metrics['f1-score']:<8.2f} {specificity_per_class[idx]:<12.2f} {accuracy_per_class[idx]:<8.2f} {metrics['support']:<8}")

    # Print overall accuracy
    print(f"\nOverall Accuracy: {overall_accuracy:.2f}")

    # Macro and Micro average metrics
    macro_precision = precision_score(true_labels, predicted_classes, average='macro', zero_division=0)
    macro_recall = recall_score(true_labels, predicted_classes, average='macro', zero_division=0)
    macro_f1 = f1_score(true_labels, predicted_classes, average='macro', zero_division=0)

    micro_precision = precision_score(true_labels, predicted_classes, average='micro', zero_division=0)
    micro_recall = recall_score(true_labels, predicted_classes, average='micro', zero_division=0)
    micro_f1 = f1_score(true_labels, predicted_classes, average='micro', zero_division=0)

    print("\nMacro Average Metrics:")
    print(f"Precision: {macro_precision:.2f}, Recall: {macro_recall:.2f}, F1-Score: {macro_f1:.2f}, Specificity: {macro_specificity:.2f}")

    print("\nMicro Average Metrics:")
    print(f"Precision: {micro_precision:.2f}, Recall: {micro_recall:.2f}, F1-Score: {micro_f1:.2f}, Specificity: {micro_specificity:.2f}")

evaluate_model(true_labels, predicted_classes, class_labels, save_dir)


In [ ]:
def plot_confusion_matrix(cm, class_labels, title='Confusion Matrix', cmap=plt.cm.Blues, save_dir=None):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(class_labels))
    plt.xticks(tick_marks, class_labels, rotation=45)
    plt.yticks(tick_marks, class_labels)
    
    thresh = cm.max() / 2.0
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, f"{cm[i, j]}", horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    if save_dir:
        plt.savefig(f"{save_dir}/confusion_matrix.png", dpi=600, bbox_inches='tight')
    plt.show()

# ROC-AUC 
def plot_roc_auc_curves(true_labels, predictions, class_labels, save_dir):
    binarized_labels = tf.keras.utils.to_categorical(true_labels, num_classes=len(class_labels))
    plt.figure(figsize=(12, 8))
    for i in range(len(class_labels)):
        fpr, tpr, _ = roc_curve(binarized_labels[:, i], predictions[:, i])
        roc_auc = roc_auc_score(binarized_labels[:, i], predictions[:, i])
        plt.plot(fpr, tpr, lw=2, label=f'ROC Curve for class {class_labels[i]} (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC-AUC Curves')
    plt.legend(loc='lower right')
    plt.grid(True)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    plt.savefig(os.path.join(save_dir, 'roc_auc_curve.png'), dpi=600, bbox_inches='tight')
    plt.show()

# Precision-Recall
def plot_precision_recall_curves(true_labels, predictions, class_labels, save_dir):
    binarized_labels = tf.keras.utils.to_categorical(true_labels, num_classes=len(class_labels))
    plt.figure(figsize=(12, 8))
    for i in range(len(class_labels)):
        precision, recall, _ = precision_recall_curve(binarized_labels[:, i], predictions[:, i])
        pr_auc = tf.keras.metrics.AUC()(tf.convert_to_tensor(recall, dtype=tf.float32), tf.convert_to_tensor(precision, dtype=tf.float32)).numpy()
        plt.plot(recall, precision, lw=2, label=f'Precision-Recall Curve for class {class_labels[i]} (area = {pr_auc:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curves')
    plt.legend(loc='lower left')
    plt.grid(True)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    plt.savefig(os.path.join(save_dir, 'precision_recall_curve.png'), dpi=600, bbox_inches='tight')
    plt.show()

plot_confusion_matrix(cm, class_labels, title='Confusion Matrix', save_dir=save_dir)
plot_roc_auc_curves(true_labels, predictions, class_labels, save_dir)
plot_precision_recall_curves(true_labels, predictions, class_labels, save_dir)

In [ ]:
explainer = lime_image.LimeImageExplainer()

def model_predict(image_batch):
    return model.predict(image_batch)

class_labels = list(test_generator.class_indices.keys())

num_images = 1
class_images = {label: [] for label in class_labels}

for images, labels in test_generator:
    for i, label in enumerate(labels):
        label_index = np.argmax(label)
        class_name = class_labels[label_index]
        if len(class_images[class_name]) < num_images:
            class_images[class_name].append(images[i])
    if all(len(class_images[label]) >= num_images for label in class_labels):
        break
save_dir = "plots/lime"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
for class_name, images in class_images.items():
    for idx, img in enumerate(images):
        explanation = explainer.explain_instance(
            img.astype('double'),
            model_predict,
            top_labels=1,
            num_samples=1000
        )
        temp, mask = explanation.get_image_and_mask(
            explanation.top_labels[0],
            positive_only=False,
            num_features=10
        )
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.imshow(img) 
        plt.title(f"Original: {class_name}")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
        plt.title(f"Explained: {class_name} | Image {idx + 1}")
        plt.axis('off')
        
        save_path = os.path.join(save_dir, f"{class_name}_image_{idx + 1}.png")
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()


In [ ]:
class_labels = list(test_generator.class_indices.keys())

num_images = 3
class_images = {label: [] for label in class_labels}

for images, labels in test_generator:
    for i, label in enumerate(labels):
        label_index = np.argmax(label)
        class_name = class_labels[label_index]
        if len(class_images[class_name]) < num_images:
            class_images[class_name].append((images[i], label_index))  
    if all(len(class_images[label]) >= num_images for label in class_labels):
        break

save_dir = "plots/grad_cam"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

target_layer_name = [layer.name for layer in model.layers if 'conv2d' in layer.name][-1]

grad_cam = GradCAM()

for class_name, images in class_images.items():
    for idx, (img, label_index) in enumerate(images):
        data = ([img], None) 
        cam_result = grad_cam.explain(
            data, model, class_index=label_index, layer_name=target_layer_name
        )

        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.imshow(img)  
        plt.title(f"Original: {class_name}")
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(img)  
        plt.imshow(cam_result, cmap='jet', alpha=0.5) 
        plt.title(f"GradCAM: {class_name} | Image {idx + 1}")
        plt.axis('off')
        save_path = os.path.join(save_dir, f"{class_name}_image_{idx + 1}.png")
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()

In [ ]:

batch_size = 10 

test_data = []

try:
    while len(test_data) < batch_size:
        images, _ = next(test_generator)
        test_data.extend(images)
except StopIteration:
    print("Data generator is exhausted. Collected:", len(test_data), "samples.")

if len(test_data) > 0:
    test_data = np.array(test_data[:batch_size])
else:
    raise ValueError("No data available from the generator. Please check the generator configuration.")

print("Test data shape:", test_data.shape)

masker = shap.maskers.Image("inpaint_telea", test_data[0].shape)  
explainer = shap.Explainer(model, masker)  

print("Generating SHAP values...")
shap_values = explainer(test_data)  
print("SHAP values generated successfully!")

overall_shap_values = np.sum(np.abs(shap_values.values), axis=-1)  

flattened_shap_values = overall_shap_values.reshape(test_data.shape[0], -1)
flattened_test_data = test_data.reshape(test_data.shape[0], -1) 

print("Plotting SHAP summary plot...")
plt.figure(figsize=(10, 6))
shap.summary_plot(
    flattened_shap_values,
    flattened_test_data,
    plot_type="dot", 
    show=False
)
plt.title("SHAP Summary Plot (Overall)")

save_dir = "plots/shap_summary"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_path = os.path.join(save_dir, "shap_summary_overall.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.close()

print(f"Overall SHAP summary plot saved at {save_path}")

In [ ]:
# Display the first few images with true and predicted labels
for i in range(20):  # Display the first 20 images
    plt.figure()
    plt.imshow(image.load_img(os.path.join(test_dir, test_generator.filenames[i])))
    plt.title(f"True Label: {class_labels[true_labels[i]]}, Predicted Label: {class_labels[np.argmax(predictions[i])]}")
    plt.axis('off')
    plt.show()